# How Many Covid-19 Cases Are Actually Out There?

## Data Collection and Cleaning

#### Because of home testing, closing of PCR sites, and less sequencing, epidemiologists believe official Covid-19 case counts are very likely to be under-reported. 

#### Can we utilize additional data sources and machine learning to better predict what the actual case count is likely to be?

In [1]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import json
import requests

#### Warning: this notebook will not be the most interesting thing you read today.

I'm using hosptialization data and wastewater data from the CDC, variant prevalence from GISAID, and daily case, death, and testing totals by state from Johns Hopkins University. This notebook is about pulling together all the data from those seperate data sources and condensing it into a dataframe for predictive analytics.

The sources for the data are linked on my repo's main site [here](https://mattlscruggs.github.io/DataScienceRamblings/). I mention it below in the code, but the CDC's API for pulling data from their site limits you to 1000 rows. Until I can figure out a way around that, I'm reading the data from a flat file on my local machine (as you can probably tell from the path in the code chunk below) and updating manually, but of course my preference would be to read directly from a link for maximum reproducibility. 

The enventual goal is to be able to determine the relationship between our dependent variable (US covid-19 cases per day) and our independent variables (all the other stuff I just mentioned). But first, that requires a clean, properly formatted, tidy data set.

In [2]:
hosp_url="C:/Users/matth/OneDrive/Documents/Personal Docs/Data Science/Science projects/Project wildfire/actual case calculator/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_State_Timeseries.csv"
ww_1_url="C:/Users/matth/OneDrive/Documents/Personal Docs/Data Science/Science projects/Project wildfire/actual case calculator/NWSS_Public_SARS-CoV-2_Concentration_in_Wastewater_Data.csv"
ww_2_url="C:/Users/matth/OneDrive/Documents/Personal Docs/Data Science/Science projects/Project wildfire/actual case calculator/NWSS_Public_SARS-CoV-2_Wastewater_Metric_Data.csv"
testing_url="https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/testing_data/time_series_covid19_US.csv"
variant_url="https://raw.githubusercontent.com/hodcroftlab/covariants/master/cluster_tables/USAClusters_data.json"
cases_url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
deaths_url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
vacc_url="C:/Users/matth/OneDrive/Documents/Personal Docs/Data Science/Science projects/Project wildfire/actual case calculator/COVID-19_Vaccinations_in_the_United_States_County.csv"
states_lookup="C:/Users/matth/OneDrive/Documents/Personal Docs/Data Science/Science projects/Project wildfire/states.csv"

Use this file as a key to make sure we're only getting states we want.

In [3]:
states=pd.read_csv(states_lookup, names=['state_name','state_abbv'])

In [4]:
state_names=states['state_name'].unique().tolist()

In [5]:
state_abbs=states['state_abbv'].unique().tolist()

This final data set is going to be measured at at the state/day level. This gives us a bit more granularity and robustness rather than inferring a national trend by day, since we have 50x the data  points to refer to.

In [6]:
pd.set_option('display.max_rows', 900)

#### Starting with the variant data
This data set tracks the percentages of sequences belonging to each variant. They've identified a lot of variants, so I'm narrowing it down to the major ones and creating an "other" category.

In [7]:
variant_raw=json.loads(requests.get(variant_url).text)

In [8]:
state_list=pd.DataFrame.from_dict(variant_raw['countries'], orient='index').index.to_list()

In [9]:
out=pd.DataFrame()
for state in state_list:   
    tmp=pd.DataFrame.from_dict(variant_raw['countries'][state])
    tmp['state']=state
    out=pd.concat([out,tmp], ignore_index=True)

Here I'm dividing the number of sequenced cases for each variant by the total sequenced on that day.

In [10]:
out['alpha_per']=out['20I (Alpha, V1)']/out['total_sequences']
out['beta_per']=out['20H (Beta, V2)']/out['total_sequences']
out['gamma_per']=out['20J (Gamma, V3)']/out['total_sequences']
out['delta_per']=(out['21A (Delta)']+out['21I (Delta)']+out['21J (Delta)'])/out['total_sequences']
out['omicron_per']=(out['21K (Omicron)']+out['21L (Omicron)'])/out['total_sequences']
out['kappa_per']=out['21B (Kappa)']/out['total_sequences']
out['eta_per']=out['21D (Eta)']/out['total_sequences']
out['iota_per']=out['21F (Iota)']/out['total_sequences']
out['lambda_per']=out['21G (Lambda)']/out['total_sequences']
out['mu_per']=out['21H (Mu)']/out['total_sequences']
out['eu1_per']=out['20E (EU1)']/out['total_sequences']
out['epsilon_per']=out['21C (Epsilon)']/out['total_sequences']
out['week']=pd.to_datetime(out['week'])
out['other_per']=1-(out['alpha_per']+out['beta_per']+out['gamma_per']+out['delta_per']+out['omicron_per']+out['kappa_per']+\
               out['eta_per']+out['iota_per']+out['lambda_per']+out['mu_per']+out['eu1_per']+out['epsilon_per'])

In [11]:
variant_df=out[['week','state','alpha_per','beta_per','gamma_per', 'delta_per',
       'omicron_per', 'kappa_per', 'eta_per', 'iota_per', 'lambda_per',
       'mu_per', 'eu1_per', 'epsilon_per','other_per']].sort_values(by='week').fillna(0).replace([np.inf, -np.inf],0).reset_index(drop=True)

I'm selecting the columns I need above, and below I'm adding a "month" and "year" variable. The percentages are calculated every two weeks, and that's going to be a pain to join to the other data by date, so I'm averaging the percentages across the months.

Also the "month" and "year" variables are going to be important predictors, since we've seen spikes in December/January (Christmas/New Years), July (July 4th/summer vacation), and August (back to school).

In [12]:
variant_df['month']=pd.to_datetime(variant_df['week']).dt.month
variant_df['year']=pd.to_datetime(variant_df['week']).dt.year

In [13]:
variant_agg_total=variant_df.groupby(['state','month','year']).mean().reset_index()
variant_agg=variant_agg_total[variant_agg_total['state'].isin(state_names)].drop_duplicates()

#### Final variant dataset

So here's what we have. Each row has a state, month, year, and the respoective percentages for each major strain.

In [14]:
print(variant_agg.shape)
variant_agg.head()

(1251, 16)


,state,month,year,alpha_per,beta_per,gamma_per,delta_per,omicron_per,kappa_per,eta_per,iota_per,lambda_per,mu_per,eu1_per,epsilon_per,other_per
0,Alabama,1,2021,0.016484,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.005495,0.0
1,Alabama,1,2022,0.000000,0.000000,0.000000,0.004562,0.994526,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
2,Alabama,2,2021,0.209973,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.015131,0.000000,0.000000,0.0,0.030262,0.0
3,Alabama,2,2022,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
4,Alabama,3,2021,0.567669,0.006865,0.018634,0.000000,0.000000,0.0,0.0,0.024845,0.006865,0.003106,0.0,0.045603,0.0


#### Now we'll bring in daily case data from John's Hopkins
This is at the county level, but we can sum up by state.

In [15]:
cs_wide=pd.read_csv(cases_url)

In [16]:
state_and_date_total=cs_wide.drop(['UID','iso2','iso3','code3','Country_Region','Lat','Long_','Combined_Key'], axis='columns').\
melt(id_vars=['FIPS','Admin2','Province_State'], var_name='date', value_name='cases')

In [17]:
state_and_date=state_and_date_total[state_and_date_total['Province_State'].isin(state_names)].drop_duplicates()

In [18]:
long_cases=state_and_date.drop('FIPS', axis='columns').groupby(['Province_State','date']).sum().reset_index()

In [19]:
long_cases['date']=pd.to_datetime(long_cases['date'])

Again adding month and year, but also keeping date since we have a continuous sequence of dates from Jan 2020. These dates, states, month, and year will form the backbone of the final dataframe. Everything else will join to them, eventually.

In [20]:
wide_cases=long_cases.sort_values(by='date').pivot(index='date', columns='Province_State',values='cases').diff()

In [21]:
wide_cases_roll=wide_cases.rolling(8, min_periods=1).mean().reset_index()

Rolling 7-day new cases, to account for differences in reporting, otherwise the within-value variance is too high

In [22]:
cases_agg=wide_cases_roll.melt(id_vars='date', value_name='cases').fillna(0).copy()
cases_agg.loc[(cases_agg.cases<0),'cases']=0

In [23]:
cases_agg['month']=pd.to_datetime(cases_agg['date']).dt.month
cases_agg['year']=pd.to_datetime(cases_agg['date']).dt.year

#### Final cases dataset

This one's much smaller, and has just the essential information.

In [24]:
cases_agg.head()

,date,Province_State,cases,month,year
0,2020-01-22,Alabama,0.0,1,2020
1,2020-01-23,Alabama,0.0,1,2020
2,2020-01-24,Alabama,0.0,1,2020
3,2020-01-25,Alabama,0.0,1,2020
4,2020-01-26,Alabama,0.0,1,2020


#### Now we'll do the exact same procedure for the deaths data. 
It's also from JHU, and the format is exactly the same as cases, so I'm literally copying the code from above, just pointing it to a different source file and updating the labels to reflect that it's recording deaths from covid.

In [25]:
cs_wide=pd.read_csv(deaths_url)

In [26]:
state_and_date=cs_wide.drop(['UID','iso2','iso3','code3','Country_Region','Lat','Long_','Combined_Key','Population'], axis='columns').\
melt(id_vars=['FIPS','Admin2','Province_State'], var_name='date', value_name='deaths')

In [27]:
long_deaths=state_and_date.drop('FIPS', axis='columns').groupby(['Province_State','date']).sum().reset_index()

In [28]:
long_deaths['date']=pd.to_datetime(long_deaths['date'])

In [29]:
wide_deaths=long_deaths.sort_values(by='date').pivot(index='date', columns='Province_State',values='deaths').diff()

In [30]:
wide_deaths_roll=wide_deaths.rolling(8, min_periods=1).mean().reset_index()

Rolling 7-day new cases, to account for differences in reporting, otherwise the within-value variance is too high

In [31]:
deaths_agg=wide_deaths_roll.melt(id_vars='date', value_name='deaths').fillna(0)
deaths_agg.loc[(deaths_agg.deaths<0),'deaths']=0

In [32]:
deaths_agg['month']=pd.to_datetime(deaths_agg['date']).dt.month
deaths_agg['year']=pd.to_datetime(deaths_agg['date']).dt.year

#### Final deaths dataset

In [33]:
deaths_agg.head()

,date,Province_State,deaths,month,year
0,2020-01-22,Alabama,0.0,1,2020
1,2020-01-23,Alabama,0.0,1,2020
2,2020-01-24,Alabama,0.0,1,2020
3,2020-01-25,Alabama,0.0,1,2020
4,2020-01-26,Alabama,0.0,1,2020


#### Now for the hospitalizations. 
This one's coming from the CDC, and for whatever reason, their API only let's me read 1000 rows from a URL. I downloaded the file on my local machine, and I'll add the link to the readme for my repo. If you want to reproduce this research, that's probably the easiest path. This is a pretty big file, so I don't want to throw the whole thing in the repo, but I'll have the source link and the final dataframe.

In [34]:
hosp_raw=pd.read_csv(hosp_url)

In [35]:
hosp_select=hosp_raw[['state','date','total_adult_patients_hospitalized_confirmed_covid','total_pediatric_patients_hospitalized_confirmed_covid']].copy()

Adding the same date/month/year variables, so everything joins up nicely and has appropriate counterparts. This dataset had a TON of different metrics for hospitalization, so I added all the confirmed adult cases and all the confirmed pediatric cases. 

I think in the future it would be cool to use the variables by age group; since the goal is to use hospitalizations to infer background cases, it seems like either some groups would land in the hospital more, and so could be predictive of a rising trend, or conversely, if cases go up dramatically after a particular age group sees an increase in hospitalizations (indicative of high total cases in that age group), then we have a potential superspreader age bracket identified.

In [36]:
hosp_select['date']=pd.to_datetime(hosp_select['date'])
hosp_select['month']=pd.to_datetime(hosp_select['date']).dt.month
hosp_select['year']=pd.to_datetime(hosp_select['date']).dt.year
hosp_select['hospitalized_covid']=hosp_select['total_adult_patients_hospitalized_confirmed_covid']+\
hosp_select['total_pediatric_patients_hospitalized_confirmed_covid']

In [37]:
hospital_agg=hosp_select.merge(states, how='inner', left_on='state', right_on='state_abbv', copy=False).drop('state', axis='columns').sort_values(by='date').fillna(0)

#### Final hospitalization dataset

Again, very similar final variables as the above data sets. I'm only bringing the total hosptializations, not adult vs pediatric at this point.

In [38]:
hospital_agg.head()

,date,total_adult_patients_hospitalized_confirmed_covid,total_pediatric_patients_hospitalized_confirmed_covid,month,year,hospitalized_covid,state_name,state_abbv
22918,2020-01-01,0.0,0.0,1,2020,0.0,Hawaii,HI
28928,2020-01-01,0.0,0.0,1,2020,0.0,Louisiana,LA
14921,2020-01-01,0.0,0.0,1,2020,0.0,North Carolina,NC
9120,2020-01-01,0.0,0.0,1,2020,0.0,Alabama,AL
15666,2020-01-01,0.0,0.0,1,2020,0.0,Minnesota,MN


#### Now for the testing data. 
Again, this is from JHU, and has tests split into viral and antigen. I've lumped them together, but I could totally see going back and looking at them seperately, in case there's a stronger signal there.

In [39]:
test_raw=pd.read_csv(testing_url).fillna(0)

In [40]:
test_raw['tested_total']=test_raw['tests_antigen_total']+test_raw['tests_viral_total']
test_raw['positive_total']=test_raw['tests_antigen_positive']+test_raw['tests_viral_positive']
test_raw['tested_daily']=test_raw['tested_total'].diff()
test_raw['positive_daily']=test_raw['positive_total'].diff()
test_raw['positive_rate']=test_raw['positive_daily']/test_raw['tested_daily']

This data is also cumulative, so the "tests_antigen_total" column is a running total of all tests done in a state since Jan 2020. I'm taking the difference between one day and the previous day, for total as well as positive, and then dividing positive tests by total tests, to get the positivity rate for the day.

In [41]:
test_raw['date']=pd.to_datetime(test_raw['date'])
test_raw['month']=pd.to_datetime(test_raw['date']).dt.month
test_raw['year']=pd.to_datetime(test_raw['date']).dt.year

Because these dates are stacked by state, we get some weird values when, for example, we subtract Wisconsin's cases on Jan 21 2020 from West Virginia's cases on Apr 22 2022; the "daily increase" is extremely negative, because the subtraction is going back in time two years and a different state. I just dropped the very few rows where this happens, so all percentage values are between 0% and 100%. 

In [42]:
test_small=test_raw[['date','month','year','state','positive_rate']].fillna(0).replace([np.inf, -np.inf],0)

In [43]:
test_agg=test_small[test_small['state'].isin(state_abbs)]

In [44]:
test_agg=test_agg[(test_agg.positive_rate <= 1) & (test_agg.positive_rate >= 0)]

#### Final testing data set

In [45]:
test_agg.head()

,date,month,year,state,positive_rate
0,2020-03-06,3,2020,AK,0.0
1,2020-03-07,3,2020,AK,0.0
2,2020-03-08,3,2020,AK,0.0
3,2020-03-09,3,2020,AK,0.0
4,2020-03-10,3,2020,AK,0.0


#### Now for wastewater monitoring. 
This is actually two files, again from the CDC, and again on my local machine since the API is again, limiting me to 1000 rows.

In [46]:
water_1=pd.read_csv(ww_1_url, low_memory=False)
water_2=pd.read_csv(ww_2_url, low_memory=False)

In [47]:
water_lookup=water_2[['key_plot_id','wwtp_jurisdiction']].drop_duplicates()

In [48]:
water=water_1.merge(water_lookup, how='inner',on='key_plot_id')

I joined the two files, since one has the state name and the water treatment site name, and the other just has the treatment site name and the concentration.

In [49]:
water['date']=pd.to_datetime(water['date'])
water['month']=pd.to_datetime(water['date']).dt.month
water['year']=pd.to_datetime(water['date']).dt.year

Because these measurements are taken irregularly, I filled the NA values with the previous valid entry; basically, the measurement doesn't change until we get a new measurement, whether that's a week, or two, or three.

In [50]:
water=water.sort_values(by="date").fillna(method='ffill').drop('key_plot_id', axis='columns').reset_index(drop=True)

There were also two different methods for how the concentration was normalized. I chose "flow-population" which accounts for the population of the area it serves. Both made sense, but this one was more complete. In the future I might go back and incorporate the other as well.

In [51]:
water_flow=water[water['normalization']=='flow-population'].drop('normalization', axis='columns').drop_duplicates()

In [52]:
water_flow.shape

(95956, 5)

#### Final wastewater dataset

In [53]:
water_flow.head()

,date,pcr_conc_smoothed,wwtp_jurisdiction,month,year
0,2022-01-13,NaN,New York,1,2022
1,2022-01-13,1.645220e+08,Oregon,1,2022
2,2022-01-13,1.645220e+08,Illinois,1,2022
4,2022-01-13,5.623477e+07,Virginia,1,2022
5,2022-01-13,1.408148e+08,Ohio,1,2022


#### Now I'm bringing in the vaccination data set. 
It's, again, from the CDC, and looks at the county and date level, so it's pretty big (~35m rows?). 

In [54]:
vacc_raw=pd.read_csv(vacc_url, low_memory=False)

Again, lots of variables here, but I'm bringing in counts and the population estimates so I can calculate percentage of people who are vaccinated.

In [55]:
vacc_small=vacc_raw[['Date','Recip_State','Administered_Dose1_Recip','Series_Complete_Yes','Booster_Doses','Census2019']].copy()

In [56]:
vacc_small.head()

,Date,Recip_State,Administered_Dose1_Recip,Series_Complete_Yes,Booster_Doses,Census2019
0,05/17/2022,TN,514191.0,454719.0,215035.0,694144.0
1,05/17/2022,TN,2782.0,2483.0,781.0,7016.0
2,05/17/2022,TX,10195.0,9118.0,3854.0,18443.0
3,05/17/2022,TX,9307.0,8082.0,2839.0,18546.0
4,05/17/2022,TX,11435.0,9936.0,3412.0,23021.0


Adding our normal date, month, year variables, and then adding together all the county data in a state.

In [57]:
vacc_small['Date']=pd.to_datetime(vacc_small['Date'])
vacc_small['month']=pd.to_datetime(vacc_small['Date']).dt.month
vacc_small['year']=pd.to_datetime(vacc_small['Date']).dt.year

In [58]:
vacc_agg=vacc_small.groupby(['Date','Recip_State','month','year']).sum().reset_index().sort_values(by='Date')

In [59]:
vacc_agg.head()

,Date,Recip_State,month,year,Administered_Dose1_Recip,Series_Complete_Yes,Booster_Doses,Census2019
0,2020-12-13,AK,12,2020,0.0,0.0,0.0,731545.0
29,2020-12-13,ND,12,2020,0.0,0.0,0.0,762062.0
30,2020-12-13,NE,12,2020,0.0,0.0,0.0,1934408.0
31,2020-12-13,NH,12,2020,0.0,0.0,0.0,1359711.0
32,2020-12-13,NJ,12,2020,0.0,0.0,0.0,8882190.0


Now dividing the counts by the census to get the percentages, and selecting the percentages.

In [60]:
vacc_agg['partial_vacc']=vacc_agg['Administered_Dose1_Recip']/vacc_agg['Census2019']
vacc_agg['full_vacc']=vacc_agg['Series_Complete_Yes']/vacc_agg['Census2019']
vacc_agg['vacc_boost']=vacc_agg['Booster_Doses']/vacc_agg['Census2019']

In [61]:
vacc_agg=vacc_agg[['Date', 'Recip_State', 'month', 'year','partial_vacc',
       'full_vacc', 'vacc_boost']]

#### Final vaccination dataset

In [62]:
vacc_agg.head()

,Date,Recip_State,month,year,partial_vacc,full_vacc,vacc_boost
0,2020-12-13,AK,12,2020,0.0,0.0,0.0
29,2020-12-13,ND,12,2020,0.0,0.0,0.0
30,2020-12-13,NE,12,2020,0.0,0.0,0.0
31,2020-12-13,NH,12,2020,0.0,0.0,0.0
32,2020-12-13,NJ,12,2020,0.0,0.0,0.0


#### Let's get the census data too, so we can bring in per 100k cases, deaths, and hospitalizations

In [63]:
state_census=vacc_small[['Recip_State','Census2019']].groupby('Recip_State').max().reset_index().copy()
state_census=state_census[state_census['Recip_State'].isin(state_abbs)]

## Now to join everything together

Here's where all the hard work pays off. What I'm looking for is a single dataframe with all the variables above which might impact case counts, all joined together by date and state, so that I can do some nifty machine learning.

The "case" column will be what I'm predicting for future values, and the other columns are there so the algorithms can discern the mathematical relationship between the variables, so that we can predict cases given values of the other variables.

That way, even as testing sites are shut down, and people are either testing at home or not at all, we can use these more robust metrics to determine a more appropriate level of caution.

Yes, I'll be doing some fun combinatorics and probability mass functions to determine actual risk, as well as an ANOVA of the effectiveness of mask orders. I also am planning on looking at the improvement in outcomes due to vaccination, and even the relative risk of the vaccines (I have all the VAERS data going back to 1999, so we can compare these vaccines to all the previous vaccines we get, as well as the risk of hospitalization or death due to covid). 

I'm personally very pro-vaccine, but I understand lots of people have hesitations, so I'm making the science as transparent as possible to help people understand and see it how I do.

In [64]:
con_df=cases_agg.merge(deaths_agg, how='left',left_on=['Province_State','date','month','year'], right_on=['Province_State','date','month','year'], copy=False).\
merge(states, how='left', left_on=['Province_State'], right_on=['state_name'], copy=False).\
merge(test_agg, how='left', left_on=['state_abbv','date','month','year'], right_on=['state','date','month','year'], copy=False).\
merge(hospital_agg, how='left',left_on=['Province_State','state_abbv','date','month','year'], right_on=['state_name','state_abbv','date','month','year'], copy=False).\
merge(vacc_agg, how='left', left_on=['state_abbv','date','month','year'], right_on=['Recip_State','Date','month','year'],copy=False).\
merge(water_flow, how='left', left_on=['Province_State','date','month','year'], right_on=['wwtp_jurisdiction','date','month','year'], copy=False).\
merge(variant_agg, how='left', left_on=['Province_State','month','year'], right_on=['state','month','year'], copy=False).\
merge(state_census, how='left',left_on='state_abbv', right_on='Recip_State', copy=False)

Let's do some cleanup, and get the columns we really need.

In [65]:
con_df.columns

Index(['date', 'Province_State', 'cases', 'month', 'year', 'deaths',
       'state_name_x', 'state_abbv', 'state_x', 'positive_rate',
       'total_adult_patients_hospitalized_confirmed_covid',
       'total_pediatric_patients_hospitalized_confirmed_covid',
       'hospitalized_covid', 'state_name_y', 'Date', 'Recip_State_x',
       'partial_vacc', 'full_vacc', 'vacc_boost', 'pcr_conc_smoothed',
       'wwtp_jurisdiction', 'state_y', 'alpha_per', 'beta_per', 'gamma_per',
       'delta_per', 'omicron_per', 'kappa_per', 'eta_per', 'iota_per',
       'lambda_per', 'mu_per', 'eu1_per', 'epsilon_per', 'other_per',
       'Recip_State_y', 'Census2019'],
      dtype='object')

In [66]:
con_clean=con_df[['date','month', 'year', 'Province_State',  'cases', 'deaths','hospitalized_covid','partial_vacc', 'full_vacc', 'vacc_boost', 'pcr_conc_smoothed',\
                 'alpha_per', 'beta_per', 'gamma_per','delta_per', 'omicron_per', 'kappa_per', 'eta_per', 'iota_per',\
                  'lambda_per', 'mu_per', 'eu1_per', 'epsilon_per', 'other_per','positive_rate', 'Census2019']]

In [67]:
con_clean=con_clean[con_clean['Province_State'].isin(state_names)].drop_duplicates()

In [68]:
con_clean.columns

Index(['date', 'month', 'year', 'Province_State', 'cases', 'deaths',
       'hospitalized_covid', 'partial_vacc', 'full_vacc', 'vacc_boost',
       'pcr_conc_smoothed', 'alpha_per', 'beta_per', 'gamma_per', 'delta_per',
       'omicron_per', 'kappa_per', 'eta_per', 'iota_per', 'lambda_per',
       'mu_per', 'eu1_per', 'epsilon_per', 'other_per', 'positive_rate',
       'Census2019'],
      dtype='object')

In [69]:
con_out=con_clean.groupby(['date','month','year','Province_State']).agg({'cases':'mean',
                                                                         'deaths':'mean',
                                                                         'hospitalized_covid':'mean',
                                                                        'partial_vacc':'mean', 
                                                                         'full_vacc':'mean',
                                                                         'vacc_boost':'mean',
                                                                         'pcr_conc_smoothed':'mean',
                                                                         'alpha_per':'mean',
                                                                         'beta_per':'mean', 
                                                                         'gamma_per':'mean', 
                                                                         'delta_per':'mean',
                                                                         'omicron_per':'mean', 
                                                                         'kappa_per':'mean', 
                                                                         'eta_per':'mean', 
                                                                         'iota_per':'mean', 
                                                                         'lambda_per':'mean',
                                                                          'mu_per':'mean', 
                                                                         'eu1_per':'mean', 
                                                                         'epsilon_per':'mean', 
                                                                         'other_per':'mean',
                                                                        'positive_rate':'mean',
                                                                        'Census2019':'max'}).reset_index().fillna(0)

Now we need to turn daily cases into cases per 100k. Same with deaths and hospitalizations. We're already looking at variants and vaccinations as a percentage, and positivity rate as a percentage of all tests done, so it makes sense to normalize by population for the state. 

If you didn't, you could get weird results, like 100% test positivity rate leads to 100 cases in Montana and 100,000 cases in New York, and that would break the mathematical relationship. If we normalize for population we'd expect to see a similar cases per 100k people (or deaths per 100k, or hospitalizations per 100k), provided there is a mathematical relationship there.

In [70]:
con_out['daily_new_cases_per_100k']=(con_out['cases']/con_out['Census2019'])*100000
con_out['daily_new_deaths_per_100k']=(con_out['deaths']/con_out['Census2019'])*100000
con_out['daily_new_hosp_per_100k']=(con_out['hospitalized_covid']/con_out['Census2019'])*100000

Ok, that's a wrap. We have a dataframe with 41k observations of 24 variables, and one with some blank values, which has 121k observations of 24 variables. We'll see if the missing values throw off the algorithm, but if they don't, I'd rather use the more complete one, as it has a longer time frame. The issue is that the CDC only started monitoring wastewater in December 2021, so any values before that time are blank. We'll see what we can get.

In [71]:
con_out.to_csv('consolidated data- daily case count.csv', index=False)

I'm writing the final data set to a csv file, and those will be located in the repo for reference and reproducibility. They're small enough that you could probably open them in Excel if you really wanted to.

## To Be Continued...